<a href="https://colab.research.google.com/github/josephpro21/bot/blob/main/engbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mounting drive

In [5]:
from google.colab import drive
drive.mount("/content/drive")
data_path = "/content/drive/MyDrive/boteng"

Mounted at /content/drive


# Imports

In [6]:
import json
import string
import random
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

# Import glob

In [7]:
import glob
data_files = glob.glob(data_path + "/dataset.json")

with open(data_files[0], 'r') as file:
  data = json.load(file)

print(data)
#for data_file in data_files:
#  data = json.load(open(data_file))
#  print(data)

{'intents': [{'tag': 'greating', 'patterns': ['Hi', 'Hey', 'Greatings', 'How are you'], 'responses': ['Am fine how about you', 'Fine how may i help you']}, {'tag': 'advice', 'patterns': ['how may i stay healthy', 'what health tips do you have for me', 'what should i eat during pregnancy'], 'responses': ['stay hydrated']}]}


# x and y

In [8]:
all_words = []
classes = []
data_x = []
data_y = []

for intent in data["intents"]:
  for pattern in intent["patterns"]:
    tokens = nltk.wordpunct_tokenize(pattern)
    all_words.extend(tokens)
    data_x.append(pattern)
    data_y.append(intent["tag"]),

  if intent["tag"] not in classes:
    classes.append(intent["tag"])

lemmatizer = WordNetLemmatizer()

all_words = [lemmatizer.lemmatize(word.lower()) for word in all_words if word not in string.punctuation]
all_words = sorted(set(all_words))
classes = sorted(set(classes))

# Show

In [ ]:
print(len(classes))
print(data_x)
print(data_y)
print(all_words)

2
['Hi', 'Hey', 'Greatings', 'How are you', 'how may i stay healthy', 'what health tips do you have for me', 'what should i eat during pregnancy']
['greating', 'greating', 'greating', 'greating', 'advice', 'advice', 'advice']
['are', 'do', 'during', 'eat', 'for', 'greatings', 'have', 'health', 'healthy', 'hey', 'hi', 'how', 'i', 'may', 'me', 'pregnancy', 'should', 'stay', 'tip', 'what', 'you']


In [9]:
train = []
output = [0] * len(classes)

classes  = sorted(set(data_y))

for idx, doc in enumerate(data_x):
  bow = []
  text = lemmatizer.lemmatize(doc.lower())
  for word in all_words:
    bow.append(1) if word in text else bow.append(0)
  output_row = list(output)
  output_row[classes.index(data_y[idx])] = 1
  train.append([bow, output_row])

random.shuffle(train)
train = np.array(train, dtype=object)
x_train = np.array(list(train[:, 0]))
y_train = np.array(list(train[:, 1]))

# model

In [15]:
#model = Sequential()
#model.add(Dense(128, input_shape=(len(x_train[0]),), activation="relu"))
#model.add(Dropout(0.5))
#model.add(Dense(64, activation="relu"))
#model.add(Dropout(0.5))
#model.add(Dense(len(y_train[0]), activation="softmax"))
'''adam = tf.keras.optimizers.Adam(learning_rate=0.01)
def compile_model(loss,optimizer, metrics):
  model.compile(loss=loss,
                optimizer=optimizer,
                metrics=metrics)
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=["accuracy"])
print(model.summary())
model.fit(x_train, y_train, epochs=32, batch_size=8, verbose=1)
model.save("/content/model/chatbot_model.h5")
model.evaluate(x_train, y_train)
print("Done")
'''

'adam = tf.keras.optimizers.Adam(learning_rate=0.01)\ndef compile_model(loss,optimizer, metrics):\n  model.compile(loss=loss,\n                optimizer=optimizer,\n                metrics=metrics)\nmodel.compile(loss=\'categorical_crossentropy\',\n              optimizer=adam,\n              metrics=["accuracy"])\nprint(model.summary())\nmodel.fit(x_train, y_train, epochs=32, batch_size=8, verbose=1)\nmodel.save("/content/model/chatbot_model.h5")\nmodel.evaluate(x_train, y_train)\nprint("Done") \n'

In [11]:
model = tf.keras.models.load_model("/content/model/chatbot_model.h5")

In [12]:
def clean_text(text):
  tokens = nltk.wordpunct_tokenize(text)
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  return tokens
def bag_of_words(text, vocab):
  tokens = clean_text(text)
  bow = [0] * len(vocab)
  for w in tokens:
    for idx, word in enumerate(vocab):
      if word == w:
        bow[idx] = 1
  return np.array(bow)
def predict_class(text, vocab, labels):
  bow = bag_of_words(text, vocab)
  result = model.predict(np.array([bow]))[0]
  thresh = 0.2
  y_pred = [[idx, res] for idx, res in enumerate(result) if res > thresh]
  y_pred.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in y_pred:
    return_list.append({'intent': labels[r[0]], 'probability': r[1]})
  return return_list

def get_response(intents_list, intents_json):
  if len(intents_list) == 0:
    result = "Sorry! I don't Comprehende."
  else:
    tag = intents_list[0]
    list_of_intents = intents_json["intents"]
    for i in list_of_intents:
      if i["tag"] == tag:
        result = random.choice(i["responses"])
        break
      else:
        result = "Sorry! I don't Comprehende."
  return result

In [13]:
def pred_class(sentence,words,classes):
  sentence_words = nltk.word_tokenize(sentence)
  sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
  bag = [1 if word in all_words else 0 for word in sentence_words]
  return np.array(bag)

def getting_response(intents_list, data):
  # Check if intents_list is empty to avoid IndexError
  if not intents_list:
    return "Sorry! I don't Comprehende."

  # intents_list[0] is already a dictionary with 'intent' as a key
  tag = intents_list[0]['intent']

  list_of_intents = data["intents"]
  result = ""
  for i in list_of_intents:
    if i["tag"] == tag:  # Compare with 'tag' in your data
      result = random.choice(i["responses"])
      break
  return result

In [14]:
print("Press 7 to quit the bot")
while True:
  prompt = input("Enter Prompt: \t")
  if prompt == "7":
    break
  intents = predict_class(prompt, all_words, classes)
  result = getting_response(intents, data)
  print(result)

Press 7 to quit the bot
Enter Prompt: 	hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
Am fine how about you
Enter Prompt: 	7


# API

In [ ]:
from flask import Flask, request, jsonify
import tensorflow as tf
loaded_model = model
metrics = loaded_model.evaluate(x_train, y_train)
loss = metrics[0]*100
accuracy = metrics[1]*100
print("Loss:", metrics[0]*100)
print("Accuracy:", metrics[1]*100)
app = Flask(__name__)
@app.route('/prediction', methods = ['POST'])
def prediction():
  prompt = request.get_json()
  user_prompt = prompt['prompt']
  intents = predict_class(user_prompt, all_words, classes)
  result = getting_response(intents,data)
  return jsonify({"response": result, "accuracy": accuracy, "loss": loss})

if __name__ == "__main__":
  app.run()

In [3]:
import tensorflow
import flask
import nltk
import numpy as np
print(tensorflow.__version__)
print(flask.__version__)
print(nltk.__version__)
print(np.__version__)


2.17.1
3.1.0
3.9.1
1.26.4


<ipython-input-3-ab0be2592e48>:6: DeprecationWarning: The '__version__' attribute is deprecated and will be removed in Flask 3.1. Use feature detection or 'importlib.metadata.version("flask")' instead.
  print(flask.__version__)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 301ms/step - accuracy: 1.0000 - loss: 6.1477e-06
Loss: 0.0006147718067950336
Accuracy: 100.0
